In [1]:
import cv2,os,time,re
os.sys.path.append('/home/prateek/shared_space/Notebooks/abg/openvino/Notebooks/utilities/')

In [2]:
from datetime import datetime
import numpy as np
import IPython.display as Disp
import support_utility_openvino
import matplotlib.pyplot as plt
from shapely.geometry import Polygon,Point
import ipywidgets as widgets
from itertools import combinations
from sklearn.metrics import pairwise_distances
import scipy as sp

In [3]:
person_vehicle_model_path = "/home/prateek/prateek_space/intel_model_dump/intel/pedestrian-and-vehicle-detector-adas-0001/INT8/pedestrian-and-vehicle-detector-adas-0001.xml"
helmet_vest_model_path = "/media/prateek/prateek_space/model_files/openvino_model/2020_02_04_only_person_model/converted_1/frozen_inference_graph.xml"#ssd inception net model
attr_model_path = "/home/prateek/prateek_space/intel_model_dump/intel/vehicle-attributes-recognition-barrier-0039/INT8/vehicle-attributes-recognition-barrier-0039.xml"

In [4]:
output_support = support_utility_openvino.create_plot()

In [5]:
num_requests = 2

In [6]:
person_vehicle_model = support_utility_openvino.async_infer(num_requests=num_requests)
person_vehicle_model.load_model(model_path=person_vehicle_model_path,device="MULTI:CPU,GPU")

Available Devices :  ['CPU', 'GNA', 'GPU']
model inputs : dict_keys(['data'])
model outputs :  dict_keys(['detection_out'])


In [7]:
helmet_vest_model = support_utility_openvino.async_infer(num_requests=num_requests,ie_network=person_vehicle_model.ie)
helmet_vest_model.load_model(model_path=helmet_vest_model_path,device="MULTI:CPU,GPU")

Available Devices :  ['CPU', 'GNA', 'GPU']
model inputs : dict_keys(['image_tensor'])
model outputs :  dict_keys(['DetectionOutput'])


In [8]:
attr_model = support_utility_openvino.async_infer(num_requests=num_requests,ie_network=person_vehicle_model.ie)
attr_model.load_model(model_path=attr_model_path,device="MULTI:CPU,GPU")

Available Devices :  ['CPU', 'GNA', 'GPU']
model inputs : dict_keys(['input'])
model outputs :  dict_keys(['color', 'type'])


In [9]:
def point_intersection_shapely(polygon, point):
    """
    polygon : set of points for polygon[[x1,y1],[x2,y2]] 
    point : set of points for polygon[x3,y3] 
    returns true or false
    """
    p1 = Polygon(polygon)
    return p1.contains(Point(point))


def poly_intersection_shapely(pt1,pt2,intersection_threshold=.5):
    """
    pt1 : set of points for polygon 1 [[x1,y1],[x2,y2]] 
    pt2 : set of points for polygon 2[[x1,y1],[x2,y2]] 
    intersection theshold : intersection threshold for polygon intersection considered with ref to pt2
    bool : return True or False
    """

    p1 = Polygon(pt1)
    p2 = Polygon(pt2)
    intersection_area = p1.intersection(p2).area
    if intersection_area/p2.area>intersection_threshold:
        return True
    return False


In [15]:
def get_closest_comb(bbox1,bbox2):
    pts1 = np.array([[bbox1[0],bbox1[3]],[bbox1[2],bbox1[3]]])
    pts2 = np.array([[bbox2[0],bbox2[3]],[bbox2[2],bbox2[3]]])
    D = pairwise_distances(pts1,pts2,metric='euclidean')
    coords = np.unravel_index(D.argmin(), D.shape)
    return np.array([pts1[coords[0]],pts2[coords[1]]])

def create_distance_arrow(frame, st_pt,end_pt,text,font=cv2.FONT_HERSHEY_SIMPLEX,font_scale=.8,font_thickness=2,font_color=(0,0,0),line_color=(255,255,255),display_category = False):
    """
    frame on which distance matrix have to be created
    st_pt : start point of line
    end_pt : end point of line
    text : text to write
    
    """
    
    (text_width, text_height) = cv2.getTextSize(text, font, fontScale=font_scale, thickness=font_thickness)[0]
    if display_category:
        ## assuming distance in meter and text is distance
        distance = int(re.findall('\d+',text)[0] )
        if distance<100:
            line_color = (0,0,255)
        elif distance<200:
            line_color = (0,255,255)
        else :
            line_color = (0,255,0)
    op_frame = cv2.line(frame,tuple(st_pt),tuple(end_pt),line_color,font_thickness)
    op_frame = cv2.putText(op_frame, text,tuple(((st_pt+end_pt)/2+np.array([0,text_height])).astype(int)) , font,  font_scale, font_color, font_thickness, cv2.LINE_AA) 
    return op_frame
    

def write_distance_frame(frame,bboxes,M,units ="cm"):
    """
    frame : frame on which distance have to be write
    bboxes :  bbox  array
    M : Transformation matrix
    """
    combs = list(combinations(list(range(bboxes.shape[0])),2))
    for idx,comb in enumerate(combs):
        pts_comb= get_closest_comb(bboxes[comb[0]],bboxes[comb[1]])
        pts_comb_trans = cv2.perspectiveTransform(np.float32([pts_comb]),M).squeeze()
        dist = int(sp.spatial.distance.euclidean(pts_comb_trans[0],pts_comb_trans[1]))
        text = str(dist) 
        if units is not None:
            text = text + units
        frame = create_distance_arrow(frame,pts_comb[0],pts_comb[1],text,display_category = True)
    return frame

In [16]:
def combine_results(frame,res_person,bbox,hv_results,thresh_hv=.5):
    """
    frame :  frame for which results are generated 
    res_person :  person_detection model result
    bbox : person detection bbox
    res_hv : helmet vest model result
    thresh_hv : helmet vest threshold
    """
    res_person[:,1] = 6
    res_all = []
    for idx,res_hv in enumerate(hv_results):
        res_hv = res_hv[0]
        res_hv = res_hv[np.where(res_hv[:,:,:,2]>thresh_hv)]
        if res_hv.shape[0]>0:
                ##[1,2] helmet,no_helmet category
                ##[3,4,5] vest,no_vest,worker 
            h_data = res_hv[np.isin(res_hv[:,1], [1,2])]
            v_data = res_hv[np.isin(res_hv[:,1], [3,4,5])]
            res_filt = []
            try:
                ## finding max for each category since there's only one person so one entry each category
                if h_data.shape[0]!=0:
                    res_filt.append(h_data[np.argmax(h_data[:,2])])
                if v_data.shape[0]!=0:
                    res_filt.append(v_data[np.argmax(v_data[:,2])])

            except Exception as e:
                pass
                #                 print(e)
                            ####changing the bbox value from local to global
            res_filt = np.array(res_filt)
            b = bbox[idx]
            width,height = b[2]-b[0] ,b[3]-b[1] 
            res_filt[:,3:] = [width,height,width,height]*res_filt[:,3:]+[b[0],b[1],b[0],b[1]]
            res_filt[:,3:]/=np.array([frame.shape[1],frame.shape[0],frame.shape[1],frame.shape[0]])
            res_all.append(res_filt)  
    # pdb.set_trace()

    res_all.append(res_person)
    res_all = np.concatenate(res_all)
    return res_all

In [17]:

class bbox_select():
    %matplotlib widget 


    def __init__(self,im):
        self.im = im
        self.selected_points = []
        self.fig,ax = plt.subplots()
        self.img = ax.imshow(self.im.copy())
        self.ka = self.fig.canvas.mpl_connect('button_press_event', self.onclick)
        disconnect_button = widgets.Button(description="Disconnect mpl")
        Disp.display(disconnect_button)
        disconnect_button.on_click(self.disconnect_mpl)


        
    def poly_img(self,img,pts):
        pts = np.array(pts, np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.polylines(img,[pts],True,(0,255,255),4)
        return img

    def onclick(self, event):
    #display(str(event))
        self.selected_points.append([event.xdata,event.ydata])
        if len(self.selected_points)>1:
            self.fig
            self.img.set_data(self.poly_img(self.im.copy(),self.selected_points))
    def disconnect_mpl(self,_):
        self.fig.canvas.mpl_disconnect(self.ka)

        
            

In [27]:
channel = "/home/prateek/prateek_space/helmet_n_vest/test_data/2019_09_12_Taloja_video_data/2019-09-12/annealing_view/comb_output.mp4"
cap = cv2.VideoCapture(channel)

In [28]:
ret,frame = cap.read()

bs = bbox_select(frame)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

In [29]:
bs.selected_points

[[652.4354366179434, 1059.7580467962448],
 [1380.1773721018144, 614.5967564736642],
 [1693.7257591985885, 626.2096596994706],
 [1349.2096301663305, 1063.6290145381804]]

In [30]:
width = 450
height = 2320


final_points = [[0,0],[width,0],[width,height],[0,height]]


M = cv2.getPerspectiveTransform(np.float32(bs.selected_points),np.float32(final_points))

In [39]:
channel = "/home/prateek/prateek_space/helmet_n_vest/test_data/2019_09_12_Taloja_video_data/2019-09-12/annealing_view/comb_output.mp4"
cap = cv2.VideoCapture(channel)
thresh_person = .3
res_range = [.002,.03]
intersection_thresh = .1
show_bbox = True
fps_async = []
f_name = 1
while True:
    t0 = time.time()
    ret,frame = cap.read()
    if not ret:
        break
    person_vehicle_model.predict(frame)
    if person_vehicle_model.frame_processed>=person_vehicle_model.num_requests:
        frame,attr,res = person_vehicle_model.postprocess_op()
        
        op,bboxes,res_filt = output_support.trim_frame_with_result(frame,res[0],threshold=thresh_person,return_results = True,resolution_thresh_range = res_range)
        for im in op:
            cv2.imwrite(str(f_name).zfill(5)+"_.jpg",im)
            f_name+=1
        if len(bboxes)>1:
        #         D =pairwise_distances(cv2.perspectiveTransform(np.float32([bboxes[:,2:]]),M).squeeze()).astype(int)
            frame = write_distance_frame(frame,np.array(bboxes),M)
        
        if show_bbox:
            frame = cv2.polylines(frame,[np.reshape(bs.selected_points,(-1,1,2)).astype('int32')],True,(255,255,0),2,cv2.LINE_AA)
        for b in bboxes:
            if poly_intersection_shapely(bs.selected_points,[[b[0],b[1]],[b[2],b[1]],[b[2],b[3]],[b[0],b[3]]],intersection_threshold=intersection_thresh)and(point_intersection_shapely(bs.selected_points,[b[0],b[3]]) or point_intersection_shapely(bs.selected_points,[b[2],b[3]])):
                cv2.rectangle(frame, (b[0], b[1]), (b[2], b[3]), (0,0,255), 2)            
            else:
                # if not intersecting
                #pass
                cv2.rectangle(frame, (b[0], b[1]), (b[2], b[3]), (0,0,0), 2)
        fps_async.append(1/(time.time()-t0))
        frame = output_support.write_text(frame,"FPS :%.2f"%np.median(fps_async),text_color=(0, 0, 0),font_scale=1)
        cv2.imshow("output",frame)
        k = cv2.waitKey(1)
        if k==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

        

    
    
    
    
        
    

In [40]:
b

[413, 68, 442, 104]

In [41]:
bboxes

[[413, 68, 442, 104]]